In [10]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [11]:
from datasets.modelnet40.modelnet40_loader import get_dataloaders, ModelNet40
path = "../data/ModelNet40/pointclouds"
categories = ['bottle']
te = ModelNet40(path, categories=categories)

Loading data: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 858.17category/s]


In [12]:
from utils.visualization import visualize_notebook
import torch

t = torch.stack([te[i]['pc'] for i in range(32)])
visualize_notebook(t, x_offset=2.5, y_offset=2.5, point_size=0.025)

Output()

In [21]:
from models import *

m = SPVD_L()

lr = 1e-4
model = DiffusionBase(m, lr=lr)

model_state_dict = torch.load("../checkpoints/ModelNet40/all/epoch+1=00-val_loss=0.10.ckpt", weights_only=True)['state_dict']
model.load_state_dict(model_state_dict)
model = model.cuda().eval()

from utils.schedulers import DDPMSparseSchedulerGPU

ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, pres=1e-5)

In [22]:
preds = ddpm_sched.sample(model.cuda(), 16, 2048)
visualize_notebook(preds, x_offset=2.5, y_offset=2.5, point_size=0.025)

Output()